In [1]:
from tkinter import *
from tkinter import ttk, messagebox
import tkinter as tk
import datetime as dt
from mydb import Database
from salary import SalaryDatabase  # Import SalaryDatabase from salary module
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from alert_system import update_last_entry_time

# Object for expense database
data = Database(db='expense.db')

# Object for salary database
salary_data = SalaryDatabase(db='Salary.db')

# Global variables
count = 0
selected_rowid = 0
salvar = salary_data.get_last_salary()  # Initialize salvar with the last saved salary
balance_alert_threshold = salvar * (20 / 100)
AF=('Arial', 15) #common font used

# Functions
def saveRecord():
    global data
    data.insertRecord(item_name=item_name.get(), item_price=item_amt.get(), purchase_date=transaction_date.get())
    update_last_entry_time()  # Update the last entry time
    check_balance_alert()
    refreshData()

def setDate():
    date = dt.datetime.now()
    dopvar.set(f'{date:%d %B %Y}')

def clearEntries():
    item_name.delete(0, 'end')
    item_amt.delete(0, 'end')
    transaction_date.delete(0, 'end')

def fetch_records():
    global count
    count = 0  # Reset count to avoid incrementing indefinitely
    f = data.fetchRecord('select rowid, * from expense_record')
    for rec in f:
        tv.insert(parent='', index='end', iid=count, values=(rec[0], rec[1], rec[2], rec[3]))
        count += 1

def select_record(event):
    global selected_rowid
    selected = tv.focus()
    val = tv.item(selected, 'values')
    try:
        selected_rowid = val[0]
        d = val[3]
        namevar.set(val[1])
        amtvar.set(val[2])
        dopvar.set(str(d))
    except Exception as ep:
        pass

def update_record():
    global selected_rowid

    selected = tv.focus()
    if not selected_rowid:
        messagebox.showwarning("Update Error", "No item selected.")
        return

    try:
        data.updateRecord(namevar.get(), amtvar.get(), dopvar.get(), selected_rowid)
        refreshData()
    except Exception as ep:
        messagebox.showerror('Error', ep)

    # Clear entry boxes
    clearEntries()

def totalBalance():
    global salvar
    f = data.fetchRecord(query="SELECT SUM(item_price) FROM expense_record")
    total_expense = f[0][0] if f and f[0][0] is not None else 0
    
    if salvar is not None:
        if total_expense:
            balance_remaining = salvar - total_expense
            messagebox.showinfo('Current Balance:', f"Total Expense: {total_expense} \nBalance Remaining: {balance_remaining}")
        else:
            messagebox.showinfo('Current Balance:', f"Total Expense: {total_expense} \nBalance Remaining: {salvar}")
    else:
        messagebox.showinfo('Current Balance:', f"Total Expense: {total_expense} \nBalance Remaining: N/A (Salary not set)")

    check_balance_alert()

def refreshData():
    children = tv.get_children()
    for child in children:
        tv.delete(child)
    fetch_records()

def deleteRow():
    global selected_rowid
    if not selected_rowid:
        messagebox.showwarning("Deletion Error", "No item selected.")
        return

    try:
        data.removeRecord(selected_rowid)
        selected_rowid = 0  # Reset selected_rowid after deletion
        refreshData()
    except Exception as ep:
        messagebox.showerror("Error", f"An error occurred while deleting the record: {ep}")

def salRec():
    global salvar
    try:
        entered_salary = float(Salary_recieved.get())  #Converting salary to float datatype
        salvar = entered_salary
        salary_data.insert_salary(entered_salary)  #Salary is inserted into database
        messagebox.showinfo("Salary Saved", f"Salary saved: {salvar}")
    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter a valid number for salary")

def plot_expense_pie_chart():
    f = data.fetchRecord('select item_name, item_price from expense_record')
    items = [rec[0] for rec in f]
    prices = [rec[1] for rec in f]

    if not items or not prices:
        messagebox.showerror("No Data", "No expense data available to plot")
        return

    # Create a new top-level window for the pie chart
    chart_window = tk.Toplevel(ws)
    chart_window.title("Expense Distribution Chart")
    chart_window.geometry("600x600")

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.pie(prices, labels=items, autopct='%1.1f%%', startangle=140)
    ax.set_title("Expense Distribution")

    # Embed the plot in the new top-level window
    canvas = FigureCanvasTkAgg(fig, master=chart_window)
    canvas.draw()
    canvas.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)

    def close_chart(event):
        chart_window.destroy()
        plt.close(fig)

    # Bind the close event
    canvas.get_tk_widget().bind("<Button-1>", close_chart)



def check_balance_alert():
    global salvar
    f = data.fetchRecord(query="SELECT SUM(item_price) FROM expense_record")
    total_expense = f[0][0] if f and f[0][0] is not None else 0
    
    if salvar is not None:
        if total_expense:
            balance_remaining = salvar - total_expense
            if balance_remaining < balance_alert_threshold:
                alert_graphics()

def alert_graphics():
    alert_window = tkk.Toplevel(ws)
    alert_window.title("Alert")
    alert_window.geometry("650x150")
    label = tkk.Label(alert_window, text="Warning: Amount is lower than 20% of the Salary!", fg="red", font=("Helvetica", 19))
    label.pack(pady=20)
    button = tkk.Button(alert_window, text="OK", command=alert_window.destroy)
    button.pack(pady=10)

def show_suggestion(age, salvar):
    suggestion_text = ""
    if 0 <= age <= 12:
        if salvar > 0:
            suggestion_text = "TOO YOUNG!!! Let your parents handle your finance"

        
    elif 13 <= age <= 19:
        if 0 <= salvar <= 50000:
            suggestion_text = "Open a Savings Account: Start saving part of pocket money or part-time job earnings.\n" \
                              "Set Short-Term Goals: Save for small purchases like gadgets or outings.\n" \
                              "Budgeting: Track income and expenses to develop good habits."
    elif 20 <= age <= 30:
        if 0 < salvar <= 30000:
            suggestion_text = "Emergency Fund: Save 3-6 months of living expenses.\n" \
                              "Retirement Savings: Start a PPF or NPS, contribute small amounts regularly.\n" \
                              "Debt Management: Focus on paying off high-interest debts."
        elif salvar > 30000:
            suggestion_text = "Investing: Begin with low-cost mutual funds or SIPs.\n" \
                              "Save for a House: Start saving for a down payment in a high-interest savings account or FD."
    elif 31 <= age <= 50:
        if 0 < salvar <= 35000:
            suggestion_text = "Increase Retirement Contributions: Maximize PPF or NPS contributions.\n" \
                              "Children's Education Savings: Open a Sukanya Samriddhi Yojana (for daughters) or a 529 plan equivalent.\n" \
                              "Diversify Investments: Include stocks, bonds, and real estate."
        elif salvar > 35000:
            suggestion_text = "Life Insurance: Ensure adequate coverage for family protection.\n" \
                              "Estate Planning: Create or update a will, consider setting up a trust."
    elif 51 <= age <= 65:
        if 0 < salvar <= 70000 :
            suggestion_text = "Catch-Up Contributions: Utilize higher limits for retirement accounts like EPF or NPS.\n" \
                              "Review Investment Portfolio: Adjust based on risk tolerance and retirement timeline.\n" \
                              "Health Savings: Consider health insurance and Maximize HSA equivalents."
        elif salvar > 70000:
            suggestion_text = "Pay Down Debt: Focus on eliminating remaining debts, including home loan.\n" \
                              "Retirement Plan: Develop a detailed strategy for income and expenses."
    elif age >= 66:
        if 0 < salvar <= 50000:
            suggestion_text = "Pension Planning: Optimize pension schemes like EPF, NPS.\n" \
                              "Income Streams: Ensure diverse sources (rent, annuities, investments)."
        elif salvar > 50000:
            suggestion_text = "Minimize Taxes: Work with a financial advisor to reduce tax burdens.\n" \
                              "Healthcare Planning: Budget for healthcare and potential long-term care.\n" \
                              "Estate Planning: Regularly update your estate plan to reflect changes."
    else:
        suggestion_text="The age and salary given has no suggestions ready, will be available soon!!"
    
    return suggestion_text

def show_suggestions_frame():
    global suggestion_label
    age = int(Age.get())
    suggestion_text = show_suggestion(age, salvar)
    suggestion_label.config(text=suggestion_text)
    suggestion_frame.pack(fill=BOTH, expand=True)

def hide_suggestions_frame():
    suggestion_frame.pack_forget()

# GUI Setup
ws = Tk()
ws.title('Expenditure Tracking')
ws['bg'] = '#0A3453'

f1 = Frame(ws, padx=10, pady=10, bg='#0A3453')
f1.pack(fill=X)
f2 = Frame(ws, bg='#0A3453')
f2.pack(fill=X)
f3 = Frame(ws, bg='#0A3453')
f3.pack(fill=BOTH, expand=True)

namevar = StringVar()
amtvar = IntVar()
dopvar = StringVar()

# Labels
Label(f1, text='ITEM NAME', font=(AF), bg='#0A3453', fg='white').grid(row=0, column=0, sticky=W)
Label(f1, text='ITEM PRICE', font=(AF), bg='#0A3453', fg='white').grid(row=1, column=0, sticky=W)
Label(f1, text='PURCHASE DATE', font=(AF), bg='#0A3453', fg='white').grid(row=2, column=0, sticky=W, pady=(0, 5))
Label(f1, text='SALARY RECEIVED', font=(AF), bg='#0A3453', fg='white').grid(row=3, column=0, sticky=W)
Label(f1, text='AGE FOR SUGGESTION', font=(AF), bg='#0A3453', fg='white').grid(row=4, column=0, sticky=W, pady=(5, 5))

# Entry widgets
item_name = Entry(f1, font=(AF), textvariable=namevar)
item_amt = Entry(f1, font=(AF), textvariable=amtvar)
transaction_date = Entry(f1, font=(AF), textvariable=dopvar)
Salary_recieved = Entry(f1, font=(AF))
Age = Entry(f1, font=(AF))

# Entry grid placement
item_name.grid(row=0, column=1, sticky=EW, padx=(10, 0))
item_amt.grid(row=1, column=1, sticky=EW, padx=(10, 0))
transaction_date.grid(row=2, column=1, sticky=EW, padx=(10, 0), pady=(0, 5))
Salary_recieved.grid(row=3, column=1, sticky=EW, padx=(10, 0))
Age.grid(row=4, column=1, sticky=EW, padx=(10, 0), pady=(5, 5))

# Action buttons
cur_date = Button(f1, text='Current Date', font=(AF), bg='#ADFF2F', command=setDate, width=15)
submit_btn = Button(f1, text='Save Record', font=(AF), command=saveRecord, bg='#7FFFD4', fg='black')
clr_btn = Button(f1, text='Clear Entry', font=(AF), command=clearEntries, bg='#FF6347', fg='black')
quit_btn = Button(f1, text='Exit', font=(AF), command=lambda: ws.destroy(), bg='#FF6347', fg='black')
total_bal = Button(f1, text='Total Balance', font=(AF), bg='#ADFF2F', command=totalBalance)
update_btn = Button(f1, text='Update', bg='#FF6347', command=update_record, font=(AF))
del_btn = Button(f1, text='Delete', bg='#FF6347', command=deleteRow, font=(AF))
sal_btn = Button(f1, text='SALARY', bg='#7FFFD4', command=salRec, font=(AF))
pie_chart_btn = Button(f1, text='Plot Pie Chart', bg='#ADFF2F', command=plot_expense_pie_chart, font=(AF))
show_suggestions_btn = Button(f1, text='Show Suggestions', font=(AF), command=show_suggestions_frame, bg='#FFD700', fg='saddle brown')
hide_suggestions_btn = Button(f1, text='Hide Suggestions', font=(AF), command=hide_suggestions_frame, bg='#FFD700', fg='saddle brown')

# New grid placement for buttons (starting from row 5)
del_btn.grid(row=5, column=0, sticky=EW, padx=(10, 0))
update_btn.grid(row=6, column=0, sticky=EW, padx=(10, 0))
clr_btn.grid(row=7, column=0, sticky=EW, padx=(10, 0))
cur_date.grid(row=5, column=1, sticky=EW, padx=(10, 0))
total_bal.grid(row=6, column=1, sticky=EW, padx=(10, 0))
pie_chart_btn.grid(row=7, column=1, sticky=EW, padx=(10, 0))
submit_btn.grid(row=5, column=2, sticky=EW, padx=(10, 0))
sal_btn.grid(row=6, column=2, sticky=EW, padx=(10, 0))
quit_btn.grid(row=7, column=2, sticky=EW, padx=(10, 0))
show_suggestions_btn.grid(row=6, column=3, sticky=EW, padx=(10, 0))
hide_suggestions_btn.grid(row=7, column=3, sticky=EW, padx=(10, 0))



# Treeview widget
tv = ttk.Treeview(f2, columns=(1, 2, 3, 4), show='headings', height=15)
tv.pack(side=LEFT)
tv.heading(1, text='ID')
tv.heading(2, text='Item Name')
tv.heading(3, text='Item Price')
tv.heading(4, text='Purchase Date')

style = ttk.Style()
style.theme_use("clam")
style.map("Treeview")

# Map colors for Treeview widget
style.map("Treeview",
          background=[('selected', '#BA55D3')])

# Map colors for Treeview headings
style.map("Treeview.Heading",
          background=[('active', '#800080')],
          foreground=[('active', 'white')])


# Scrollbar for treeview
scrollbar = ttk.Scrollbar(f2, orient=VERTICAL, command=tv.yview)
tv.configure(yscroll=scrollbar.set)
scrollbar.pack(side=RIGHT, fill=Y)

# Bind the select event to the treevieww
tv.bind('<<TreeviewSelect>>', select_record)

# Frame for suggestions
suggestion_frame = Frame(ws, bg='#FF69B4')
suggestion_label = Label(suggestion_frame , text="", font=(AF), bg='#FF69B4', fg='white', justify=LEFT)
suggestion_label.pack(pady=20, padx=20)

# Initialize data fetching
fetch_records()
#ws.after(0, check_for_alert) # For notification system
check_balance_alert() # To check if balance is below threshold value
# Main loop
ws.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\nihal\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\nihal\AppData\Local\Temp\ipykernel_19924\1493042349.py", line 227, in show_suggestions_frame
    age = int(Age.get())
ValueError: invalid literal for int() with base 10: ''
